# Imports




In [10]:
from google.colab import drive
drive.mount('/content/drive')
!pip install ipynb
import sys
sys.path.insert(0,'/content/drive/MyDrive/CNN-deterministic-decoupling')
path = '/content/drive/MyDrive/CNN-deterministic-decoupling'
import numpy as np
import torch
from ipynb.fs.defs.imageFunctions import prepare_image,show_image,save_image
from ipynb.fs.defs.DISTS_mod import DISTS
from PIL import Image
import matplotlib.pyplot as plt
import math

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Hacia atras Adam

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
target = prepare_image(Image.open(path+"/imagenes/texture6.jpg").convert("RGB"),size=128)
target=target.to(device)
target.requires_grad_(False)

#random=torch.normal(mean=0.45, std=0.22,size=target.size())
origin=prepare_image(Image.open(path+"/imagenes/texture5.jpg").convert("RGB"),size=128)
origin=origin.to(device)
origin.requires_grad_(True)
show_image(origin)
show_image(target)

x0=origin.detach().clone()
x0=x0.to(device)

ErrorList=[]
EpochList=[]
LossList=[]

model_d = DISTS().to(device)
model_d.set_target(target)
count = 0

ErrorList=[math.sqrt(((x0-origin)**2).mean())/math.sqrt((x0**2).mean())]
EpochList=[count]
initial_loss=model_d.forward(origin).item()
LossList=[initial_loss]
count+=1

optimizer = torch.optim.Adam([{"params": [origin],"lr": 0.01}])

while LossList[-1] > 0.1:
    optimizer.zero_grad()
    loss=model_d.forward(origin)
    loss.backward()
    optimizer.step()

    error=math.sqrt(((x0-origin)**2).mean())/math.sqrt((x0**2).mean())
    ErrorList.append(error)
    EpochList.append(count)
    LossList.append(loss.item())
    count+=1
    if count%2000==0:
      print("iteracion "+str(count))
      print("loss "+str(LossList[-1]))
      show_image(origin)
      with open(path+'/Datos/Adam_data_r_6_5.txt', 'w') as f:
        f.write(f'Total iterations: {count}\n')
        f.write('\n')
        f.write(f'Error values: {ErrorList}\n')
        f.write('\n')
        f.write(f'Loss values: {LossList}\n')

print("END 1")
print("iteracion "+str(count))
save_image(origin,path+'/Datos/adam_turning_6_5.png')
last_error=ErrorList[-1]

model_d.set_target(x0)
error_count=0
error_margin=60

show_image(origin)
while error_count < error_margin:

  if ErrorList[-1] > last_error:
    error_count += 1
  else:
    error_count = 0

  optimizer.zero_grad()
  loss=model_d(origin)
  loss.backward()
  optimizer.step()
  error=math.sqrt(((x0-origin)**2).mean())/math.sqrt((x0**2).mean())

  last_error=ErrorList[-1]
  ErrorList.append(error)
  EpochList.append(count)
  LossList.append(loss.item())
  count+=1

  if count%2000==0:
    print("iteracion "+str(count))
    show_image(origin)
    with open(path+'/Datos/Adam_data_r_6_5.txt', 'w') as f:
      f.write(f'Total iterations: {count}\n')
      f.write('\n')
      f.write(f'Error values: {ErrorList}\n')
      f.write('\n')
      f.write(f'Loss values: {LossList}\n')
    save_image(origin,path+'/Datos/adam_current_6_5.png')

print("END2")
print("iteracion "+str(count))



In [ ]:
plt.plot(EpochList,ErrorList)
plt.grid(True)
print(ErrorList[-1])

In [ ]:
plt.plot(EpochList,LossList)
plt.grid(True)

In [ ]:
plt.plot(EpochList[:200],LossList[:200])
plt.grid(True)

In [ ]:
plt.plot(LossList,ErrorList)
plt.grid(True)

In [ ]:
show_image(origin)